In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.datasets import mnist
import pandas as pd 
BATCH_SIZE = 64
WEIGHT_DECAY = 0.001
LEARNING_RATE = 0.001


In [2]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_images = os.getcwd() + "/train_images/" + train_df.iloc[:, 0].values
test_images = os.getcwd() + "/test_images/" + test_df.iloc[:, 0].values

train_labels = train_df.iloc[:, 1:].values
test_labels = test_df.iloc[:, 1:].values


def read_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_image(image, channels=1, dtype=tf.float32)

    # In older versions you need to set shape in order to avoid error
    # on newer (2.3.0+) the following 3 lines can safely be removed
    image.set_shape((64, 64, 1))
    label[0].set_shape([])
    label[1].set_shape([])

    labels = {"first_num": label[0], "second_num": label[1]}
    return image, labels
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = (
    train_dataset.shuffle(buffer_size=len(train_labels))
    .map(read_image)
    .batch(batch_size=BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = (
    test_dataset.map(read_image)
    .batch(batch_size=BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)

In [3]:
inputs = keras.Input(shape = (64, 64, 1))
x = layers.Conv2D(
    filters = 32,
    kernel_size = 3,
    padding = 'same',
    kernel_regularizer = keras.regularizers.l2(WEIGHT_DECAY),
)(inputs)
x = layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = layers.Conv2D(64, 3, kernel_regularizer = keras.regularizers.l2(WEIGHT_DECAY))(x)
x = layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(64, 3, activation = 'relu', kernel_regularizer = keras.regularizers.l2(WEIGHT_DECAY))(x)
x = layers.Conv2D(128, 3, activation = 'relu')(x)
x = layers.MaxPooling2D()(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation = 'relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(64, activation = 'relu')(x)
output1 = layers.Dense(10, activation='softmax', name='first_num')(x)
output2 = layers.Dense(10, activation='softmax', name='second_num')(x)

In [4]:
model = keras.Model(inputs=inputs, outputs = [output1,output2])
model.compile(
    optimizer = keras.optimizers.Adam(LEARNING_RATE),
    loss = [keras.losses.SparseCategoricalCrossentropy(),keras.losses.SparseCategoricalCrossentropy()],
    metrics=["accuracy"],
)

In [5]:
model.fit(train_dataset, epochs=5, verbose=2)

Epoch 1/5
1000/1000 - 33s - loss: 1.8521 - first_num_loss: 0.8871 - second_num_loss: 0.8794 - first_num_accuracy: 0.6878 - second_num_accuracy: 0.6897 - 33s/epoch - 33ms/step
Epoch 2/5
1000/1000 - 29s - loss: 0.5845 - first_num_loss: 0.2580 - second_num_loss: 0.2595 - first_num_accuracy: 0.9140 - second_num_accuracy: 0.9147 - 29s/epoch - 29ms/step
Epoch 3/5
1000/1000 - 29s - loss: 0.3835 - first_num_loss: 0.1640 - second_num_loss: 0.1598 - first_num_accuracy: 0.9459 - second_num_accuracy: 0.9476 - 29s/epoch - 29ms/step
Epoch 4/5
1000/1000 - 29s - loss: 0.2904 - first_num_loss: 0.1205 - second_num_loss: 0.1150 - first_num_accuracy: 0.9622 - second_num_accuracy: 0.9630 - 29s/epoch - 29ms/step
Epoch 5/5
1000/1000 - 29s - loss: 0.2380 - first_num_loss: 0.0941 - second_num_loss: 0.0922 - first_num_accuracy: 0.9702 - second_num_accuracy: 0.9706 - 29s/epoch - 29ms/step


In [6]:
model.evaluate(test_dataset, verbose=2)

313/313 - 4s - loss: 1.2830 - first_num_loss: 0.4105 - second_num_loss: 0.8224 - first_num_accuracy: 0.8884 - second_num_accuracy: 0.7919 - 4s/epoch - 12ms/step


[1.2830489873886108,
 0.41047778725624084,
 0.822367787361145,
 0.8883500099182129,
 0.791949987411499]